# Import 

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy

In [2]:
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate BratDirReader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read eHOST annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


# Define sampling functions

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

### Define CRF Wrapper

In [10]:
from CRFWrapper import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel
from ALSampler import SamplingSimulator, ModelSamplingSimulator, RandomSamplingSimulator

In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Route', 'Dosage', 'Duration', 'Drug', 'ADE', 'Form', 'Reason', 'Frequency', 'Strength'}


In [12]:
crf_model=CRFModel(anno_types=annos)
r_simulator=RandomSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)

In [13]:
scores=r_simulator.simulate_rounds(boostrap_times=200)

2024-03-21 11:54:40.926 | INFO     | ALSampler:simulate_rounds:35 - simulate round 0.
2024-03-21 11:54:44.668 | DEBUG    | CRFWrapper:fit:251 - Reset and train CRF model...
2024-03-21 11:55:40.195 | DEBUG    | CRFWrapper:fit:259 - Training complete.
2024-03-21 11:55:40.295 | DEBUG    | CRFWrapper:bootstrap_eval:280 - Predicting eval docs...
2024-03-21 11:56:17.287 | DEBUG    | CRFWrapper:bootstrap_eval:284 - Calculate scores from bootstrapping 200 times
2024-03-21 12:00:39.376 | DEBUG    | CRFWrapper:bootstrap_eval:293 - complete
2024-03-21 12:00:39.516 | DEBUG    | ALSampler:simulate_rounds:44 - {'precision': 0.9435403132345506, 'recall': 0.7792371047764345, 'f1': 0.8535492442836575}
2024-03-21 12:00:39.518 | INFO     | ALSampler:simulate_rounds:35 - simulate round 1.
2024-03-21 12:00:46.249 | DEBUG    | CRFWrapper:fit:251 - Reset and train CRF model...
2024-03-21 12:02:40.092 | DEBUG    | CRFWrapper:fit:259 - Training complete.
2024-03-21 12:02:40.267 | DEBUG    | CRFWrapper:bootstra

In [16]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [18]:
pd.options.display.float_format='{:,.5f}'.format

In [19]:
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.94354,0.94023,0.94722,0.77924,0.77152,0.78596,0.85355,0.84848,0.85821
1,0.94329,0.94022,0.94628,0.80892,0.80119,0.81500,0.87095,0.86591,0.87517
2,0.94297,0.93944,0.94554,0.82462,0.81823,0.83039,0.87983,0.87584,0.88378
3,0.94331,0.93995,0.94607,0.83339,0.82730,0.83949,0.88495,0.88076,0.88887
4,0.94357,0.94041,0.94629,0.84271,0.83687,0.84897,0.89029,0.88649,0.89463
5,0.94570,0.94255,0.94851,0.84489,0.83914,0.85075,0.89245,0.88881,0.89659
6,0.94593,0.94288,0.94866,0.84981,0.84425,0.85546,0.89530,0.89135,0.89906
7,0.94461,0.94150,0.94742,0.85417,0.84831,0.85987,0.89711,0.89352,0.90095
8,0.94567,0.94266,0.94856,0.85806,0.85262,0.86329,0.89974,0.89619,0.90357
9,0.94589,0.94309,0.94887,0.85853,0.85240,0.86406,0.90009,0.89629,0.90338


## Bootstrap 10 runs

In [13]:
boostrap_runs=3
total_round=10

In [14]:
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [15]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [16]:
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos)
    m_simulator=RandomSamplingSimulator(train_docs, total_round=total_round, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)
    scores=m_simulator.simulate_rounds()
    all_scores.append(scores) 

2024-03-22 21:39:25.752 | INFO     | __main__:<module>:3 - start run 0.
2024-03-22 21:39:25.754 | INFO     | ALSampler:simulate_rounds:45 - simulate round 0.
2024-03-22 21:39:25.757 | INFO     | ALSampler:keep_sample:39 - current sampled: 30
2024-03-22 21:47:17.349 | INFO     | ALSampler:simulate_rounds:45 - simulate round 1.
2024-03-22 21:47:17.354 | INFO     | ALSampler:keep_sample:39 - current sampled: 60
2024-03-22 21:56:27.687 | INFO     | ALSampler:simulate_rounds:45 - simulate round 2.
2024-03-22 21:56:27.690 | INFO     | ALSampler:keep_sample:39 - current sampled: 90
2024-03-22 22:06:14.709 | INFO     | ALSampler:simulate_rounds:45 - simulate round 3.
2024-03-22 22:06:14.711 | INFO     | ALSampler:keep_sample:39 - current sampled: 120
2024-03-22 22:16:55.941 | INFO     | ALSampler:simulate_rounds:45 - simulate round 4.
2024-03-22 22:16:55.942 | INFO     | ALSampler:keep_sample:39 - current sampled: 150
2024-03-22 22:30:06.919 | INFO     | ALSampler:simulate_rounds:45 - simulate

In [17]:
joblib.dump(all_scores, r'..\data\vb_sampling\ner_rand_scores.joblib')

['..\\data\\vb_sampling\\ner_rand_scores.joblib']

In [20]:
len(all_scores[0])

10